# Простой проход батчнорма без двух обучаемых параметров

In [109]:
import numpy as np
import torch
import torch.nn as nn

def custom_batch_norm1d(input_tensor, eps):
    normed_tensor = (
        (input_tensor - input_tensor.mean(dim=0)) / 
        (input_tensor.var(dim=0, unbiased=False) + eps).sqrt()
    )
    return normed_tensor


input_tensor = torch.Tensor([[0.0, 0, 1, 0, 2], [0, 1, 1, 0, 10]])
batch_norm = nn.BatchNorm1d(input_tensor.shape[1], affine=False)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
import numpy as np
all_correct = True
for eps_power in range(10):
    eps = np.power(10., -eps_power)
    batch_norm.eps = eps
    batch_norm_out = batch_norm(input_tensor)
    custom_batch_norm_out = custom_batch_norm1d(input_tensor, eps)

    all_correct &= torch.allclose(batch_norm_out, custom_batch_norm_out)
    all_correct &= batch_norm_out.shape == custom_batch_norm_out.shape
print(all_correct)

True


# Докидываем пару параметров

In [110]:
import torch
import torch.nn as nn

input_size = 7
batch_size = 5
input_tensor = torch.randn(batch_size, input_size, dtype=torch.float)

eps = 1e-3

def custom_batch_norm1d(input_tensor, weight, bias, eps):
    normed_tensor = (
        (input_tensor - input_tensor.mean(dim=0)) / 
        (input_tensor.var(dim=0, unbiased=False) + eps).sqrt()
    ) * weight + bias
    return normed_tensor

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# batch_norm = nn.BatchNorm1d(input_size, eps=eps)
# batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
# batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
# batch_norm_out = batch_norm(input_tensor)
# custom_batch_norm_out = custom_batch_norm1d(input_tensor, batch_norm.weight.data, batch_norm.bias.data, eps)
# print(torch.allclose(batch_norm_out, custom_batch_norm_out) \
#       and batch_norm_out.shape == custom_batch_norm_out.shape)

True


In [277]:
import torch
import random
import numpy as np

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

# Докидываем возможность инференса

In [300]:
import torch
import torch.nn as nn


input_size = 3
batch_size = 5
eps = 1e-1


class CustomBatchNorm1d:
    def __init__(self, weight, bias, eps, momentum):
        self.weight = weight
        self.bias = bias
        self.eps = eps
        self.momentum = momentum
        
        self.running_mean = torch.zeros_like(weight)
        self.running_var = torch.ones_like(weight)
        
        self.is_eval = False

    def __call__(self, input_tensor):
        if self.is_eval:
            mu = self.running_mean
            var = self.running_var
        else:
            mu = input_tensor.mean(dim=0)
            var = input_tensor.var(dim=0, unbiased=False)
            var_unb = input_tensor.var(dim=0, unbiased=True)
            batch_size = input_tensor.shape[0]
        
        normed_tensor = ((input_tensor - mu) / (var + self.eps).sqrt()) * self.weight + self.bias
        
        if not self.is_eval:            
            self.running_mean = (1 - self.momentum) * mu + self.momentum * self.running_mean
            self.running_var = (1 - self.momentum) * var_unb + self.momentum * self.running_var
            
        
        return normed_tensor

    def eval(self):
        self.is_eval = True
        

batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5

custom_batch_norm1d = CustomBatchNorm1d(batch_norm.weight.data,
                                        batch_norm.bias.data, eps, batch_norm.momentum)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-06) \
        and norm_output.shape == custom_output.shape

batch_norm.eval()
custom_batch_norm1d.eval()

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-06) \
        and norm_output.shape == custom_output.shape

print(all_correct)

True


# Докидываем возможность нескольких слоев
## В этот раз снова выкидываем гамму и бету

In [362]:
import torch
import torch.nn as nn

eps = 1e-3

input_channels = 3
batch_size = 3
height = 10
width = 10
# batch_size = 2
# height = 3
# width = 4

batch_norm_2d = nn.BatchNorm2d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, height, width, dtype=torch.float)


def custom_batch_norm2d(input_tensor, eps):
#     normed_tensor = torch.zeros_like(input_tensor)
#     for channel_idx in range(input_tensor.shape[1]):
#         normed_tensor[:,channel_idx,:,:] = (
#             (input_tensor[:,channel_idx,:,:] - input_tensor[:,channel_idx,:,:].mean()) / 
#             (input_tensor[:,channel_idx,:,:].var(unbiased=False) + eps).sqrt()
#             # (input_tensor[:,channel_idx,:,:] - input_tensor[:,channel_idx,:,:].mean(dim=0)) / 
#             # (input_tensor[:,channel_idx,:,:].var(dim=0, unbiased=False) + eps).sqrt()
#         )
        
    mean = input_tensor.mean(dim=(0,2,3), keepdim=True)
    var = input_tensor.var(dim=(0,2,3), unbiased=False, keepdim=True)
    # var = (input_tensor-mean).pow(2).mean(dim=(0,2,3), keepdim=True)
    normed_tensor = (input_tensor-mean)/torch.sqrt(var+eps)
    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
norm_output = batch_norm_2d(input_tensor)
custom_output = custom_batch_norm2d(input_tensor, eps)
print(torch.allclose(norm_output, custom_output) and norm_output.shape == custom_output.shape)

True


# Нормализация по каналу - внутри картинки исправляем контрастность!
## В этот раз снова выкидываем гамму и бету

In [393]:
import torch
import torch.nn as nn

eps = 1e-10

def custom_layer_norm(input_tensor, eps):
    dims = list(range(len(input_tensor.shape)))[1:]
    mean = input_tensor.mean(dim=dims, keepdim=True)
    var = input_tensor.var(dim=dims, unbiased=False, keepdim=True)
    # var = (input_tensor-mean).pow(2).mean(dim=dims, keepdim=True)
    normed_tensor = (input_tensor-mean)/torch.sqrt(var+eps)
    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True
for dim_count in range(3, 9):
    input_tensor = torch.randn(*list(range(3, dim_count + 2)), dtype=torch.float)
    layer_norm = nn.LayerNorm(input_tensor.size()[1:], elementwise_affine=False, eps=eps)

    norm_output = layer_norm(input_tensor)
    custom_output = custom_layer_norm(input_tensor, eps)

    all_correct &= torch.allclose(norm_output, custom_output, 1e-2)
    all_correct &= norm_output.shape == custom_output.shape
print(all_correct)

True


# Нормализация по инстансу
## В этот раз снова выкидываем гамму и бету

In [527]:
import torch
import torch.nn as nn

eps = 1e-3

batch_size = 5
input_channels = 2
input_length = 4

instance_norm = nn.InstanceNorm1d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, input_length, dtype=torch.float)


def custom_instance_norm1d(input_tensor, eps):
    mean = input_tensor.mean(dim=2, keepdim=True)
    var = input_tensor.var(dim=2, unbiased=False, keepdim=True)
    # var = (input_tensor-mean).pow(2).mean(dim=2, keepdim=True)
    normed_tensor = (input_tensor-mean)/torch.sqrt(var+eps)
    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
norm_output = instance_norm(input_tensor)
custom_output = custom_instance_norm1d(input_tensor, eps)
print(torch.allclose(norm_output, custom_output, atol=1e-06) and norm_output.shape == custom_output.shape)

True


In [530]:
input_tensor

tensor([[[-6.0567e-01,  4.9202e-01, -7.3778e-01,  1.1460e+00],
         [-9.2655e-01,  1.3959e+00, -9.2169e-01, -1.1080e-01]],

        [[ 1.9162e-01, -2.5859e-01, -4.8185e-01, -1.6269e-01],
         [-5.8354e-01,  1.9507e-01,  2.3379e+00, -2.6748e-01]],

        [[-6.6755e-01,  8.7841e-01, -1.2619e+00, -4.1741e-04],
         [-2.2297e-01,  2.4580e-01,  5.4644e-01,  5.2114e-01]],

        [[ 3.0778e-01, -3.9558e-01,  5.3197e-01,  1.4432e-01],
         [-1.2614e+00, -4.4672e-01, -1.7875e-01, -4.7632e-01]],

        [[ 1.5018e-01, -1.3405e+00,  9.8265e-03,  3.0408e-01],
         [ 8.1366e-01, -1.7668e+00,  7.8070e-01, -1.6876e+00]]])

In [529]:
input_tensor.mean(dim=2, keepdim=True)

tensor([[[ 0.0736],
         [-0.1408]],

        [[-0.1779],
         [ 0.4205]],

        [[-0.2629],
         [ 0.2726]],

        [[ 0.1471],
         [-0.5908]],

        [[-0.2191],
         [-0.4650]]])

# Нормализация по группе
## В этот раз снова выкидываем гамму и бету

In [534]:
import torch
import torch.nn as nn

eps = 1e-3
batch_size = 20
channel_count = 6
input_size = 2

input_tensor = torch.randn(batch_size, channel_count, input_size)


def custom_group_norm(input_tensor, groups, eps):
    channel_cnt = input_tensor.shape[1]
    grp_size = channel_cnt // groups
    
    normed_tensor = torch.zeros(
        input_tensor.shape[0], 
        input_tensor.shape[1], 
        input_tensor.shape[2], 
    )
    
    for grp_idx in range(groups):
        mean = input_tensor[:, grp_idx*grp_size:grp_idx*grp_size+grp_size, :].mean(dim=(1,2), keepdim=True)
        var = input_tensor[:, grp_idx*grp_size:grp_idx*grp_size+grp_size, :].var(dim=(1,2), unbiased=False, keepdim=True)
        # var = (input_tensor[:, grp_idx*grp_size:grp_idx*grp_size+grp_size, :]-mean).pow(2).mean(dim=(1,2), keepdim=True)
        normed_tensor[:, grp_idx*grp_size:grp_idx*grp_size+grp_size, :] = (input_tensor[:, grp_idx*grp_size:grp_idx*grp_size+grp_size, :]-mean)/torch.sqrt(var+eps)
    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True
for groups in [1, 2, 3, 6]:
    group_norm = nn.GroupNorm(groups, channel_count, eps=eps, affine=False)
    norm_output = group_norm(input_tensor)
    custom_output = custom_group_norm(input_tensor, groups, eps)
    all_correct &= torch.allclose(norm_output, custom_output, 1e-3)
    all_correct &= norm_output.shape == custom_output.shape
print(all_correct)

True
